In [19]:
import pandas as pd
import numpy as np

# Função dinâmica para calcular o número de CEPs a selecionar
def calcular_num_ceps_proporcional(n_ceps, total_ceps, min_ceps=3, max_ceps=5):
    # Calcular a proporção de CEPs na cidade em relação ao total
    proporcao = n_ceps / total_ceps
    
    # Definir os limites de percentis
    percentis = np.linspace(0, 1, max_ceps - min_ceps + 2)[1:-1]
    
    # Calcular o número de CEPs com base nos percentis
    for i, p in enumerate(percentis):
        if proporcao <= p:
            return min_ceps + i
    
    return max_ceps

# Carregar dados em um DataFrame
df = pd.read_csv("/home/mage/main/product-data-shelf-crw/packages/zip_code_brazil.csv", encoding='ISO-8859-1')
df.columns = ["id", "cidade", "logradouro", "bairro", "cep", "rua"]

# Função para selecionar CEPs proporcionalmente
def selecionar_ceps(grupo):
    # Ordenar por CEP
    grupo = grupo.sort_values(by='cep')
    
    # Número total de CEPs na cidade
    n_ceps = len(grupo)
    
    # Calcular o número de CEPs a selecionar usando a função dinâmica
    num_ceps = calcular_num_ceps_proporcional(n_ceps, len(df))
    
    # Selecione os CEPs de forma distribuída pelos percentis
    if num_ceps > 1:
        percentis = np.linspace(0, 100, num_ceps + 2)[1:-1]
        indices_selecionados = np.percentile(np.arange(n_ceps), percentis).astype(int)
    else:
        indices_selecionados = [0]  # Caso tenha apenas 1 CEP disponível
    
    return grupo.iloc[indices_selecionados]

# Aplicar a função para cada cidade
ceps_selecionados = df.groupby('cidade').apply(selecionar_ceps).reset_index(drop=True)
ceps_selecionados

/tmp/ipykernel_204772/2677006424.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/mage/main/product-data-shelf-crw/packages/zip_code_brazil.csv", encoding='ISO-8859-1')
/tmp/ipykernel_204772/2677006424.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ceps_selecionados = df.groupby('cidade').apply(selecionar_ceps).reset_index(drop=True)


,id,cidade,logradouro,bairro,cep,rua
0,6358,Abreu e Lima,Cento e Vinte e Seis,Caetés I,53530-110,Rua
1,10985,Abreu e Lima,do Rubídio,Planalto,53550-300,Rua
2,27601,Abreu e Lima,Santa Luzia,Alto São Miguel,53560-780,Rua
3,AC,Acre,69900,69999,NaN,NaN
4,AC,Acre,69900,69999,NaN,NaN
...,...,...,...,...,...,...
1093,123357,Várzea Paulista,Luiz Marceli,Jardim Promeca,13223-080,Rua
1094,20230,Várzea Paulista,Araucária,Jardim Alessandra,13224-615,Rua
1095,2651,Xinguara,Cruzeiro do Sul,Centro,68555-460,Rua
1096,9090,Xinguara,Prudente de Moraes,Selectas,68557-045,Rua


In [112]:
df = pd.read_csv("/home/mage/main/product-data-shelf-crw/packages/zip_code_brazil.csv", encoding='ISO-8859-1')
df.columns = ["id", "cidade", "logradouro", "bairro", "cep", "rua"]

grouped_df = df.groupby("cidade").count()
filtered_grouped_df = grouped_df[grouped_df['id'] != 1]
df = filtered_grouped_df["id"].reset_index()
df.columns = ['cidade', 'n_ceps']
df["market_per_n_cep"] = (round(df["n_ceps"] * 91000 / 628435)).astype(int)


x = 0.125
y = [round((i * x), 2) for i in range(1, 9)]
y
quartis = df['market_per_n_cep'].quantile(y)
quartis
def quartile_normalization_improved(value, quartis, thresholds=None, base_value=3, step=1):
    if thresholds is None:
        thresholds = y
    
    # Loop para comparar com os quartis calculados
    for i, threshold in enumerate(thresholds):
        if value <= quartis[threshold]:
            return base_value + i * step  # Incrementa o valor base
    
    return base_value + len(thresholds) * step  # Se maior que todos os thresholds, retorna o valor máximo

# Aplicando a função melhorada aos dados
df['market_per_n_cep_normalized_quartis_improved'] = df['market_per_n_cep'].apply(
    lambda val: quartile_normalization_improved(val, quartis)
)

df = df.sort_values("market_per_n_cep")
df = df.iloc[int(len(df) * 0.25):]



/tmp/ipykernel_204772/1834855472.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/mage/main/product-data-shelf-crw/packages/zip_code_brazil.csv", encoding='ISO-8859-1')


In [117]:
df

,cidade,n_ceps,market_per_n_cep,market_per_n_cep_normalized_quartis_improved
131,Igarassu,658,95,4
235,Planaltina de Goiás,658,95,4
119,Garanhuns,665,96,5
49,Cabo Frio,666,96,5
280,Sertãozinho,661,96,5
...,...,...,...,...
112,Fortaleza,12443,1802,10
38,Belo Horizonte,12465,1805,10
262,Salvador,15134,2191,10
258,Rio de Janeiro,21568,3123,10
